# Проект "Титанік": Прогнозування виживання пасажирів

**Мета:** Побудувати модель машинного навчання, яка прогнозує, чи вижив пасажир, на основі його даних (клас, вік, стать тощо).

## 1. Імпорт бібліотек

Імпортуємо `pandas` та `numpy` для роботи з даними, `matplotlib` та `seaborn` для візуалізації, та `sklearn` для побудови моделі.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

## 2. Завантаження та аналіз даних (EDA)

Ми використаємо вбудований набір даних 'titanic' з бібліотеки `seaborn`. Це значно спрощує завантаження.

In [ ]:
# Завантажуємо дані
data = sns.load_dataset('titanic')

# Дивимось на перші 5 рядків
print("Перші 5 рядків даних:")
print(data.head())

# Отримуємо загальну інформацію про стовпці та типи даних
print("\nІнформація про дані:")
data.info()

### 2.1. Пошук пропущених значень

Це найважливіший крок у цьому проекті. Ми бачимо, що у `age` (вік) та `deck` (палуба) багато пропусків.

In [ ]:
print("\nКількість пропущених значень:")
print(data.isnull().sum())

### 2.2. Візуалізація

Подивімося, як стать та клас каюти впливали на виживання.

In [ ]:
sns.set_theme(style="whitegrid")

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Вплив статі на виживання
sns.countplot(x='survived', hue='sex', data=data, ax=ax1)
ax1.set_title('Виживання за статтю')
ax1.set_xticklabels(['Загинув', 'Вижив'])

# Вплив класу на виживання
sns.countplot(x='survived', hue='pclass', data=data, ax=ax2)
ax2.set_title('Виживання за класом')
ax2.set_xticklabels(['Загинув', 'Вижив'])

plt.show()

## 3. Обробка та підготовка даних (Feature Engineering)

Зараз ми "очистимо" дані, щоб модель могла їх зрозуміти.

In [ ]:
# Вибираємо ознаки (features), які будемо використовувати
# 'survived' - це наша ціль (y), решта - ознаки (X)
features = ['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked']
target = 'survived'

# Створюємо копії, щоб не змінювати оригінальні дані
X = data[features].copy()
y = data[target].copy()

# --- 1. Обробка пропущених значень ---

# 'age': Заповнюємо пропущений вік медіанним (середнім) значенням
# strategy='median' - стійкий до викидів
age_imputer = SimpleImputer(strategy='median')
X['age'] = age_imputer.fit_transform(X[['age']])

# 'embarked': Заповнюємо пропущений порт посадки модою (найчастішим значенням)
embarked_imputer = SimpleImputer(strategy='most_frequent')
X['embarked'] = embarked_imputer.fit_transform(X[['embarked']])

print("Дані після заповнення пропусків:")
X.info()

# --- 2. Кодування категоріальних ознак ---

# Моделі не розуміють текст ('male', 'female', 'S', 'C', 'Q'). Треба перетворити їх на числа.

# 'sex': Перетворюємо 'male' -> 1, 'female' -> 0
X['sex'] = X['sex'].map({'male': 1, 'female': 0})

# 'embarked': Використовуємо One-Hot Encoding, щоб уникнути помилкового порядку (S > C > Q)
X = pd.get_dummies(X, columns=['embarked'], drop_first=True)

print("\nДані після кодування:")
print(X.head())

## 4. Тренування моделі

Нарешті, ми готові розділити дані та натренувати модель. Ми використаємо `RandomForestClassifier` — це потужна та надійна модель.

In [ ]:
# Розділяємо дані на тренувальну та тестову вибірки (80% / 20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Розмір тренувальної вибірки: {X_train.shape[0]}")
print(f"Розмір тестової вибірки: {X_test.shape[0]}")

# Ініціалізуємо модель
model = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=5)

# Тренуємо модель
print("\nТренування моделі...")
model.fit(X_train, y_train)
print("Модель натреновано.")

## 5. Оцінка моделі

In [ ]:
# Робимо передбачення на тестових даних
y_pred = model.predict(X_test)

# Розраховуємо точність
accuracy = accuracy_score(y_test, y_pred)
print(f"\nТочність моделі на тестовій вибірці: {accuracy * 100:.2f}%")

# Будуємо матрицю помилок
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Загинув', 'Вижив'])

fig, ax = plt.subplots(figsize=(6, 6))
disp.plot(ax=ax, cmap='Blues')
plt.title("Матриця помилок для 'Титаніка'")
plt.show()

## Висновок

Ми успішно побудували повний ML-пайплайн:
1.  Завантажили дані.
2.  Проаналізували їх та візуалізували.
3.  **Очистили дані:** заповнили пропущений вік та порт посадки, перетворили текстові дані на числові.
4.  Натренували модель `RandomForestClassifier`.
5.  Досягли точності **~80%**.

Матриця помилок показує, що модель досить добре передбачає обидва класи (і тих, хто вижив, і тих, хто загинув), хоча й припускається помилок.